Estudo Market Basket Analysis

Criado por: Caio Araujo
Data de Criação: 23/06/2024

Descrição: Estudo do Market Basket Analysis

----------------------------------------

Importação das Bibliotecas

----------------------------------------

In [1]:
#Importação
import warnings
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from mlxtend.frequent_patterns import apriori, association_rules

# Desativar todos os warnings
warnings.filterwarnings('ignore')
# Desativar todos os warnings de depreciação
warnings.filterwarnings('ignore', category=DeprecationWarning)


----------------------------------------

Definições Basicas

----------------------------------------

In [2]:
# Caminho para o arquivo Excel
file_path = 'Online_Retail.xlsx'

# Leitura do arquivo Excel
df = pd.read_excel(file_path)

In [3]:
#Visualização do Data Frame
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


----------------------------------------

Tratamento de Dados e Padronização

----------------------------------------

In [4]:
#Visualizando os tipos de dados do Data Frame.
df.dtypes

,0
InvoiceNo,object
StockCode,object
Description,object
Quantity,int64
InvoiceDate,datetime64[ns]
UnitPrice,float64
CustomerID,float64
Country,object


In [5]:
# Renomear colunas específicas para o Portugues
df = df.rename(columns={'InvoiceNo': 'cd_fatura', 'StockCode': 'cd_produto', 'Description': 'ds_produto', 'Quantity': 'vl_quantidade','InvoiceDate': 'dt_fatura','UnitPrice': 'vl_produto_unitario','CustomerID': 'cd_cliente', 'Country': 'ds_pais' })


In [6]:
# Converter a coluna dt_fatura para o formato datetime
df['dt_fatura'] = pd.to_datetime(df['dt_fatura'])

# Criar novas coluna do tipo data e hora
df['dt_fatura_data'] = df['dt_fatura'].dt.date
df['dt_fatura_hora'] = df['dt_fatura'].dt.time
df['dt_ano_mes'] = df['dt_fatura'].dt.to_period('M')
df['dt_dia_semana'] = df['dt_fatura'].dt.day_name()
df['dt_hora_dia'] = df['dt_fatura'].dt.hour

In [7]:
# Dicionário de tradução
dias_semana_traducao = {
    'Monday': 'Segunda-feira',
    'Tuesday': 'Terça-feira',
    'Wednesday': 'Quarta-feira',
    'Thursday': 'Quinta-feira',
    'Friday': 'Sexta-feira',
    'Saturday': 'Sábado',
    'Sunday': 'Domingo'
}

# Traduzir dias da semana
df['dt_dia_semana'] = df['dt_dia_semana'].apply(lambda x: dias_semana_traducao[x])

In [8]:
#Visualizar a quantidade de linhas e colunas do data frame
df.shape

(541909, 13)

In [9]:
#Visualizar informações basicas sobre o data frame
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 13 columns):
 #   Column               Non-Null Count   Dtype         
---  ------               --------------   -----         
 0   cd_fatura            541909 non-null  object        
 1   cd_produto           541909 non-null  object        
 2   ds_produto           540455 non-null  object        
 3   vl_quantidade        541909 non-null  int64         
 4   dt_fatura            541909 non-null  datetime64[ns]
 5   vl_produto_unitario  541909 non-null  float64       
 6   cd_cliente           406829 non-null  float64       
 7   ds_pais              541909 non-null  object        
 8   dt_fatura_data       541909 non-null  object        
 9   dt_fatura_hora       541909 non-null  object        
 10  dt_ano_mes           541909 non-null  period[M]     
 11  dt_dia_semana        541909 non-null  object        
 12  dt_hora_dia          541909 non-null  int32         
dtypes: datetime64[

In [10]:
#Breve descritivo estatístico do data frame
df.describe()

,vl_quantidade,dt_fatura,vl_produto_unitario,cd_cliente,dt_hora_dia
count,541909.000000,541909,541909.000000,406829.000000,541909.000000
mean,9.552250,2011-07-04 13:34:57.156386048,4.611114,15287.690570,13.078729
min,-80995.000000,2010-12-01 08:26:00,-11062.060000,12346.000000,6.000000
25%,1.000000,2011-03-28 11:34:00,1.250000,13953.000000,11.000000
50%,3.000000,2011-07-19 17:17:00,2.080000,15152.000000,13.000000
75%,10.000000,2011-10-19 11:27:00,4.130000,16791.000000,15.000000
max,80995.000000,2011-12-09 12:50:00,38970.000000,18287.000000,20.000000
std,218.081158,NaN,96.759853,1713.600303,2.443270


In [11]:
#Visualizando a quantidade de campos nulos por coluna
df.isnull().sum()

,0
cd_fatura,0
cd_produto,0
ds_produto,1454
vl_quantidade,0
dt_fatura,0
vl_produto_unitario,0
cd_cliente,135080
ds_pais,0
dt_fatura_data,0
dt_fatura_hora,0


In [12]:
#Removendo as linhas sem a descrição do produto
df.dropna(subset=['ds_produto'], inplace=True)

In [13]:
#Visualizando a quantidade de campos nulos por coluna
df.isnull().sum()

,0
cd_fatura,0
cd_produto,0
ds_produto,0
vl_quantidade,0
dt_fatura,0
vl_produto_unitario,0
cd_cliente,133626
ds_pais,0
dt_fatura_data,0
dt_fatura_hora,0


In [14]:
#Verificando a existência de duplicatas no data frame
df.duplicated().sum()

5268

In [15]:
#Removendo a relação de linhas duplicadas do data frame
df.drop_duplicates(inplace=True)

In [16]:
# Excluir linhas onde 'vl_quantidade' é igual ou menor que 0
df = df[df['vl_quantidade'] > 0]

In [17]:
#Breve descritivo estatístico do data frame
df.describe()

,vl_quantidade,dt_fatura,vl_produto_unitario,cd_cliente,dt_hora_dia
count,525462.000000,525462,525462.000000,392732.000000,525462.000000
mean,10.681024,2011-07-04 14:16:25.781883136,3.876109,15287.734822,13.074158
min,1.000000,2010-12-01 08:26:00,-11062.060000,12346.000000,6.000000
25%,1.000000,2011-03-28 11:36:00,1.250000,13955.000000,11.000000
50%,4.000000,2011-07-20 10:15:00,2.080000,15150.000000,13.000000
75%,11.000000,2011-10-19 11:41:00,4.130000,16791.000000,15.000000
max,80995.000000,2011-12-09 12:50:00,13541.330000,18287.000000,20.000000
std,157.399272,NaN,42.040021,1713.567773,2.442502


In [18]:
# Criar a coluna vl_venda_total
df['vl_vendas_geral'] = df['vl_quantidade'] * df['vl_produto_unitario']


In [19]:
# Filtrar apenas valores maiores que 0 para a coluna vl_venda_total
df = df[df['vl_vendas_geral'] > 0]


In [20]:
# Lista de valores a serem removidos da coluna 'cd_produto'
valores_a_remover = ['S','POST','PADS','M','gift_0001_50','gift_0001_40','gift_0001_30','gift_0001_20','gift_0001_10','DOT','DCGSSGIRL','DCGSSBOY','DCGS0076','DCGS0074','DCGS0073','DCGS0072','DCGS0071','DCGS0070','DCGS0069','DCGS0068','DCGS0067','DCGS0066P','DCGS0057','DCGS0055','DCGS0004','DCGS0003','D','CRUK','C2','BANK CHARGES','B','AMAZONFEE']

# Filtrar o DataFrame removendo as linhas com os valores especificados
df = df[~df['cd_produto'].isin(valores_a_remover)]


In [21]:
#Reodernar o Data Frame.
df = df[['dt_ano_mes', 'dt_fatura_data', 'dt_dia_semana', 'dt_fatura_hora', 'dt_hora_dia', 'cd_cliente', 'ds_pais', 'cd_fatura', 'cd_produto', 'ds_produto', 'vl_produto_unitario', 'vl_quantidade', 'vl_vendas_geral']]

In [22]:
df.head()

,dt_ano_mes,dt_fatura_data,dt_dia_semana,dt_fatura_hora,dt_hora_dia,cd_cliente,ds_pais,cd_fatura,cd_produto,ds_produto,vl_produto_unitario,vl_quantidade,vl_vendas_geral
0,2010-12,2010-12-01,Quarta-feira,08:26:00,8,17850.0,United Kingdom,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,2.55,6,15.30
1,2010-12,2010-12-01,Quarta-feira,08:26:00,8,17850.0,United Kingdom,536365,71053,WHITE METAL LANTERN,3.39,6,20.34
2,2010-12,2010-12-01,Quarta-feira,08:26:00,8,17850.0,United Kingdom,536365,84406B,CREAM CUPID HEARTS COAT HANGER,2.75,8,22.00
3,2010-12,2010-12-01,Quarta-feira,08:26:00,8,17850.0,United Kingdom,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,3.39,6,20.34
4,2010-12,2010-12-01,Quarta-feira,08:26:00,8,17850.0,United Kingdom,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,3.39,6,20.34


----------------------------------------

Análise Exploratória

----------------------------------------

In [23]:
# Contar a quantidade de clientes únicos
clientes_unicos = df['cd_cliente'].nunique()

print(f'Quantidade de clientes únicos: {clientes_unicos}')

Quantidade de clientes únicos: 4334


In [24]:
# Contar a quantidade de clientes únicos por país
clientes_unicos_por_pais = df.groupby('ds_pais')['cd_cliente'].nunique().reset_index()

# Renomear as colunas para melhor legibilidade
clientes_unicos_por_pais.columns = ['ds_pais', 'clientes_unicos']

# Ordenar o DataFrame do maior para o menor
clientes_unicos_por_pais = clientes_unicos_por_pais.sort_values(by='clientes_unicos', ascending=False)

# Criar o gráfico de barras com Plotly e adicionar rótulos de dados
fig = go.Figure(data=[go.Bar(
    x=clientes_unicos_por_pais['ds_pais'],
    y=clientes_unicos_por_pais['clientes_unicos'],
    text=clientes_unicos_por_pais['clientes_unicos'],
    textposition='auto'
)])

# Atualizar o layout do gráfico
fig.update_layout(
    title='Quantidade de Clientes Únicos por País',
    xaxis_title='País',
    yaxis_title='Clientes Únicos'
)

# Exibir o gráfico
fig.show()

In [25]:
# Ver o percentual de clientes únicos por país
clientes_unicos_por_pais = df.groupby('ds_pais')['cd_cliente'].nunique().reset_index()

# Renomear as colunas para melhor legibilidade
clientes_unicos_por_pais.columns = ['ds_pais', 'cd_cliente']

# Calcular o total de clientes únicos
total_clientes_unicos = clientes_unicos_por_pais['cd_cliente'].sum()

# Calcular a representatividade percentual de cada país
clientes_unicos_por_pais['representatividade_%'] = (clientes_unicos_por_pais['cd_cliente'] / total_clientes_unicos) * 100

# Ordenar a tabela do maior para o menor
clientes_unicos_por_pais = clientes_unicos_por_pais.sort_values(by='cd_cliente', ascending=False).reset_index(drop=True)

# Criar o gráfico de barras com Plotly
fig = px.bar(clientes_unicos_por_pais, x='ds_pais', y='representatividade_%', title='Representatividade por País',
             labels={'ds_pais': 'País', 'representatividade_%': 'Representatividade (%)'},
             text=clientes_unicos_por_pais['representatividade_%'].apply(lambda x: f'{x:.2f}%'))

# Ajustar a posição do texto no gráfico
fig.update_traces(textposition='auto')

# Exibir o gráfico
fig.show()

In [26]:
#Ajustando a Base para seguir a análise apenas com UK uma vez que ela representa 90,18% da base.
df_2 = df[df['ds_pais'] == 'United Kingdom']
df_2.head()

,dt_ano_mes,dt_fatura_data,dt_dia_semana,dt_fatura_hora,dt_hora_dia,cd_cliente,ds_pais,cd_fatura,cd_produto,ds_produto,vl_produto_unitario,vl_quantidade,vl_vendas_geral
0,2010-12,2010-12-01,Quarta-feira,08:26:00,8,17850.0,United Kingdom,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,2.55,6,15.30
1,2010-12,2010-12-01,Quarta-feira,08:26:00,8,17850.0,United Kingdom,536365,71053,WHITE METAL LANTERN,3.39,6,20.34
2,2010-12,2010-12-01,Quarta-feira,08:26:00,8,17850.0,United Kingdom,536365,84406B,CREAM CUPID HEARTS COAT HANGER,2.75,8,22.00
3,2010-12,2010-12-01,Quarta-feira,08:26:00,8,17850.0,United Kingdom,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,3.39,6,20.34
4,2010-12,2010-12-01,Quarta-feira,08:26:00,8,17850.0,United Kingdom,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,3.39,6,20.34


----------------------------------------

Análise Exploratória - Aprofundamento de Datas

----------------------------------------

In [27]:
#Contar a quantidade de Clientes únicos por hora.
clientes_unicos_por_data = df.groupby('dt_hora_dia')['cd_cliente'].nunique().reset_index()

# Renomear as colunas para melhor legibilidade
clientes_unicos_por_data.columns = ['dt_hora_dia', 'clientes_unicos']

# Ordenar o DataFrame do maior para o menor
clientes_unicos_por_data = clientes_unicos_por_data.sort_values(by='dt_hora_dia', ascending=True)

# Criar o gráfico de barras com Plotly e adicionar rótulos de dados
fig = go.Figure(data=[go.Bar(
    x=clientes_unicos_por_data['dt_hora_dia'],
    y=clientes_unicos_por_data['clientes_unicos'],
    text=clientes_unicos_por_data['clientes_unicos'],
    textposition='auto'
)])

# Atualizar o layout do gráfico
fig.update_layout(
    title='Quantidade de Clientes Únicos por hora do dia',
    xaxis_title='Hora',
    yaxis_title='Clientes Únicos'
)

# Exibir o gráfico
fig.show()

In [28]:
#Contar a quantidade de Clientes únicos por data.
clientes_unicos_por_data = df_2.groupby('dt_fatura_data')['cd_cliente'].nunique().reset_index()

# Renomear as colunas para melhor legibilidade
clientes_unicos_por_data.columns = ['dt_fatura_data', 'clientes_unicos']

# Ordenar o DataFrame do maior para o menor
clientes_unicos_por_data = clientes_unicos_por_data.sort_values(by='clientes_unicos', ascending=False)

# Criar o gráfico de barras com Plotly e adicionar rótulos de dados
fig = go.Figure(data=[go.Bar(
    x=clientes_unicos_por_data['dt_fatura_data'],
    y=clientes_unicos_por_data['clientes_unicos'],
    text=clientes_unicos_por_data['clientes_unicos'],
    textposition='auto'
)])

# Atualizar o layout do gráfico
fig.update_layout(
    title='Quantidade de Clientes Únicos por dia',
    xaxis_title='Data',
    yaxis_title='Clientes Únicos'
)

# Exibir o gráfico
fig.show()

In [29]:
#Contar a quantidade de Clientes únicos por dia da semana.
clientes_unicos_por_dia_semana = df_2.groupby('dt_dia_semana')['cd_cliente'].nunique().reset_index()

# Renomear as colunas para melhor legibilidade
clientes_unicos_por_dia_semana.columns = ['dt_dia_semana', 'clientes_unicos']

# Definir a ordem dos dias da semana em português
ordem_dias_semana = ['Segunda-feira', 'Terça-feira', 'Quarta-feira', 'Quinta-feira', 'Sexta-feira', 'Sábado', 'Domingo']
clientes_unicos_por_dia_semana['dt_dia_semana'] = pd.Categorical(clientes_unicos_por_dia_semana['dt_dia_semana'], categories=ordem_dias_semana, ordered=True)

# Ordenar o DataFrame de acordo com a ordem definida
clientes_unicos_por_dia_semana = clientes_unicos_por_dia_semana.sort_values(by='dt_dia_semana')

# Criar o gráfico de barras com Plotly e adicionar rótulos de dados
fig = go.Figure(data=[go.Bar(
    x=clientes_unicos_por_dia_semana['dt_dia_semana'],
    y=clientes_unicos_por_dia_semana['clientes_unicos'],
    text=clientes_unicos_por_dia_semana['clientes_unicos'],
    textposition='auto'
)])

# Atualizar o layout do gráfico
fig.update_layout(
    title='Quantidade de Clientes Únicos por Dia da Semana',
    xaxis_title='Dia da Semana',
    yaxis_title='Clientes Únicos'
)

# Exibir o gráfico
fig.show()

In [30]:
#Contar a quantidade de Clientes únicos por mês.
clientes_unicos_por_data = df.groupby('dt_ano_mes')['cd_cliente'].nunique().reset_index()

# Renomear as colunas para melhor legibilidade
clientes_unicos_por_data.columns = ['dt_ano_mes', 'clientes_unicos']

# Ordenar o DataFrame pelo período correto (ano e mês)
clientes_unicos_por_data = clientes_unicos_por_data.sort_values(by='dt_ano_mes')

# Converter 'ano_mes' para string para evitar problemas de serialização com Plotly
clientes_unicos_por_data['dt_ano_mes'] = clientes_unicos_por_data['dt_ano_mes'].astype(str)

# Criar o gráfico de barras com Plotly e adicionar rótulos de dados
fig = go.Figure(data=[go.Bar(
    x=clientes_unicos_por_data['dt_ano_mes'],
    y=clientes_unicos_por_data['clientes_unicos'],
    text=clientes_unicos_por_data['clientes_unicos'],
    textposition='auto'
)])

# Atualizar o layout do gráfico
fig.update_layout(
    title='Quantidade de Clientes Únicos por mês',
    xaxis_title='Data',
    yaxis_title='Clientes Únicos'
)

# Exibir o gráfico
fig.show()

#lientes_unicos_por_data.head(13)

----------------------------------------

Análise Exploratória - Aprofundamento de Vendas

----------------------------------------

In [31]:
#Preparação da base para análise de Vendas.
df_vendas_geral = df_2.copy()

#Agrupando o df para refletir a qtd total de vendas por cd_fatura.
df_vendas_geral_agrupado = df_vendas_geral.groupby(['dt_ano_mes', 'dt_fatura_data', 'dt_dia_semana', 'dt_fatura_hora', 'dt_hora_dia', 'cd_cliente', 'ds_pais', 'cd_fatura']).agg({'vl_quantidade': 'sum', 'vl_vendas_geral': 'sum'}).reset_index()

#Visualizando DF agrupado.
df_vendas_geral_agrupado.head()


,dt_ano_mes,dt_fatura_data,dt_dia_semana,dt_fatura_hora,dt_hora_dia,cd_cliente,ds_pais,cd_fatura,vl_quantidade,vl_vendas_geral
0,2010-12,2010-12-01,Quarta-feira,08:26:00,8,17850.0,United Kingdom,536365,40,139.12
1,2010-12,2010-12-01,Quarta-feira,08:28:00,8,17850.0,United Kingdom,536366,12,22.20
2,2010-12,2010-12-01,Quarta-feira,08:34:00,8,13047.0,United Kingdom,536367,83,278.73
3,2010-12,2010-12-01,Quarta-feira,08:34:00,8,13047.0,United Kingdom,536368,15,70.05
4,2010-12,2010-12-01,Quarta-feira,08:35:00,8,13047.0,United Kingdom,536369,3,17.85


In [32]:
#Quero uma matrix com anomes na linha, dt_semana na coluna e a qtd de vendas no meio. Com um color plot para enfatizar o dia com mais vendas.

# Contar a quantidade de Vendas por ano_mes e dia da semana
vendas_por_dia = df_vendas_geral_agrupado.groupby(['dt_ano_mes', 'dt_dia_semana']).size().reset_index(name='qtd_vendas')

# Criar uma tabela pivot para o heatmap
heatmap_data = vendas_por_dia.pivot(index='dt_ano_mes', columns='dt_dia_semana', values='qtd_vendas').fillna(0)

# Definir a ordem dos dias da semana em português
ordem_dias_semana = ['Segunda-feira', 'Terça-feira', 'Quarta-feira', 'Quinta-feira', 'Sexta-feira', 'Domingo']
heatmap_data = heatmap_data[ordem_dias_semana]

# Converter ano_mes para string para melhor exibição
heatmap_data.index = heatmap_data.index.astype(str)

# Criar o heatmap com Plotly
fig = px.imshow(
    heatmap_data,
    labels=dict(x="Dia da Semana", y="Ano e Mês", color="Quantidade de Vendas"),
    x=heatmap_data.columns,
    y=heatmap_data.index,
    color_continuous_scale="Reds",
    aspect="auto"
)

# Atualizar o layout do gráfico
fig.update_layout(
    title='Quantidade de Vendas por Ano e Mês e Dia da Semana',
    xaxis_title='Dia da Semana',
    yaxis_title='Ano e Mês'
)

# Exibir o gráfico
fig.show()

In [33]:
#Quero uma matrix com dt_semana na linha, dt_hora na coluna e a qtd de vendas no meio. Com um color plot para enfatizar o horario da semana com mais vendas.
# Contar a quantidade de Vendas por ano_mes e dia da semana
vendas_por_hora = df_vendas_geral_agrupado.groupby(['dt_hora_dia', 'dt_dia_semana']).size().reset_index(name='qtd_vendas')

# Criar uma tabela pivot para o heatmap
heatmap_data = vendas_por_hora.pivot(index='dt_hora_dia', columns='dt_dia_semana', values='qtd_vendas').fillna(0)

# Definir a ordem dos dias da semana em português
ordem_dias_semana = ['Segunda-feira', 'Terça-feira', 'Quarta-feira', 'Quinta-feira', 'Sexta-feira', 'Domingo']
heatmap_data = heatmap_data[ordem_dias_semana]

# Converter ano_mes para string para melhor exibição
heatmap_data.index = heatmap_data.index.astype(str)

# Criar o heatmap com Plotly
fig = px.imshow(
    heatmap_data,
    labels=dict(x="Dia da Semana", y="Horário", color="Quantidade de Vendas"),
    x=heatmap_data.columns,
    y=heatmap_data.index,
    color_continuous_scale="Reds",
    aspect="auto"
)

# Atualizar o layout do gráfico
fig.update_layout(
    title='Quantidade de Vendas por Dia da Semana e Horário',
    xaxis_title='Dia da Semana',
    yaxis_title='Horário do Dia'
)

# Exibir o gráfico
fig.show()

----------------------------------------

Análise Exploratória - Aprofundamento de Produtos

----------------------------------------

In [34]:
#Visualizando o Dataset.
df_2.head(5)

,dt_ano_mes,dt_fatura_data,dt_dia_semana,dt_fatura_hora,dt_hora_dia,cd_cliente,ds_pais,cd_fatura,cd_produto,ds_produto,vl_produto_unitario,vl_quantidade,vl_vendas_geral
0,2010-12,2010-12-01,Quarta-feira,08:26:00,8,17850.0,United Kingdom,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,2.55,6,15.30
1,2010-12,2010-12-01,Quarta-feira,08:26:00,8,17850.0,United Kingdom,536365,71053,WHITE METAL LANTERN,3.39,6,20.34
2,2010-12,2010-12-01,Quarta-feira,08:26:00,8,17850.0,United Kingdom,536365,84406B,CREAM CUPID HEARTS COAT HANGER,2.75,8,22.00
3,2010-12,2010-12-01,Quarta-feira,08:26:00,8,17850.0,United Kingdom,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,3.39,6,20.34
4,2010-12,2010-12-01,Quarta-feira,08:26:00,8,17850.0,United Kingdom,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,3.39,6,20.34


In [35]:
# Contar a quantidade de produtos únicos
produtos_unicos = df_2['cd_produto'].nunique()

print(f'Quantidade de produtos únicos: {produtos_unicos}')

Quantidade de produtos únicos: 3895


In [36]:
# Soma o total de produtos vendidos
produtos_unicos = df_2['vl_quantidade'].sum()

print(f'Quantidade de produtos vendidos: {produtos_unicos}')

Quantidade de produtos vendidos: 4639008


In [37]:
#Visualizar a representatividade de Vendas de um Produto.
# Agrupar os dados pela coluna nm_produto e somar os valores da coluna vl_quantidade
df_grouped = df_2.groupby('ds_produto')['vl_quantidade'].sum().reset_index()

# Ordenar os produtos pela quantidade total vendida
df_grouped = df_grouped.sort_values(by='vl_quantidade', ascending=False).reset_index(drop=True)

# Calcular o total de vendas
total_vendas = df_grouped['vl_quantidade'].sum()

# Calcular a representatividade percentual de cada produto
df_grouped['representatividade_%'] = (df_grouped['vl_quantidade'] / total_vendas) * 100

# Criar o gráfico de barras com Plotly
fig = px.bar(df_grouped,
             x='ds_produto',
             y='representatividade_%',
             title='Representatividade de Vendas por Produto',
             labels={'ds_produto': 'Produto', 'representatividade_%': 'Representatividade (%)'},
             text=df_grouped['representatividade_%'].apply(lambda x: f'{x:.2f}%'))

# Ajustar a posição do texto no gráfico
fig.update_traces(textposition='auto')

# Exibir o gráfico
fig.show()

In [38]:
#Visualizar os Top 10 Produtos mais vendidos.
# Agrupar os dados pela coluna nm_produto e somar os valores da coluna vl_quantidade
df_grouped = df_2.groupby('ds_produto')['vl_quantidade'].sum().reset_index()

# Ordenar os produtos pela quantidade total vendida
df_grouped = df_grouped.sort_values(by='vl_quantidade', ascending=False)

# Selecionar os 10 produtos mais vendidos
top_10_produtos = df_grouped.head(10)

# Criar o gráfico de barras com Plotly
fig = px.bar(top_10_produtos, x='ds_produto', y='vl_quantidade', title='Top 10 Produtos Mais Vendidos',
             labels={'ds_produto': 'Produto', 'vl_quantidade': 'Quantidade Vendida'},
             text=top_10_produtos['vl_quantidade'])

# Ajustar a posição do texto no gráfico
fig.update_traces(textposition='auto')

# Exibir o gráfico
fig.show()

In [39]:
#Visualizar a QTD de Produtos Vendidos no Mês Vs Receita total.
#Quantidade de Produtos Vendidos vs Valor Total das Vendas por Ano e Mês
df_v_f_anomes = df_vendas_geral_agrupado

# Agrupar os dados pela coluna dt_ano_mes e somar o valor das colunas vl_quantidade e vl_vendas_geral
df_v_f_anomes_agrupado = df_v_f_anomes.groupby('dt_ano_mes').agg({'vl_quantidade': 'sum', 'vl_vendas_geral': 'sum'}).reset_index()

# Converter 'dt_ano_mes' para string para melhor exibição
df_v_f_anomes_agrupado['dt_ano_mes'] = df_v_f_anomes_agrupado['dt_ano_mes'].astype(str)

# Criar o gráfico de barras e linhas com Plotly
fig = go.Figure()

# Adicionar as barras para vl_quantidade
fig.add_trace(go.Bar(
    x=df_v_f_anomes_agrupado['dt_ano_mes'],
    y=df_v_f_anomes_agrupado['vl_quantidade'],
    name='Quantidade Vendida',
    marker_color='blue'
))

# Adicionar a linha para vl_vendas_geral
fig.add_trace(go.Scatter(
    x=df_v_f_anomes_agrupado['dt_ano_mes'],
    y=df_v_f_anomes_agrupado['vl_vendas_geral'],
    name='Valor Total das Vendas',
    mode='lines',
    line=dict(color='red')
))

# Atualizar o layout do gráfico
fig.update_layout(
    title='Quantidade de Produtos Vendidos vs Valor Total das Vendas por Ano e Mês',
    xaxis_title='Ano e Mês',
    yaxis_title='Quantidade/Valor',
    barmode='group'
)

# Ajustar para que no gráfico tanto as colunas quanto as linhas apresentem o resultado sem casas decimais
fig.update_traces(
    hovertemplate='%{x}<br>Quantidade Vendida: %{y:.0f}<extra></extra>',
    selector=dict(type='bar')
)
fig.update_traces(
    hovertemplate='%{x}<br>Valor Total das Vendas: %{y:.0f}<extra></extra>',
    selector=dict(type='scatter')
)

# Exibir o gráfico
fig.show()


In [40]:
#Quantidade de Produtos Vendidos vs Valor Total das Vendas por Dia
# Agrupar os dados pela coluna dt_ano_mes e somar o valor das colunas vl_quantidade e vl_vendas_geral
df_v_f_anomes_agrupado = df_vendas_geral_agrupado.groupby('dt_fatura_data').agg({'vl_quantidade': 'sum', 'vl_vendas_geral': 'sum'}).reset_index()

# Converter 'dt_ano_mes' para string para melhor exibição
df_v_f_anomes_agrupado['dt_fatura_data'] = df_v_f_anomes_agrupado['dt_fatura_data'].astype(str)

# Criar o gráfico de barras e linhas com Plotly
fig = go.Figure()

# Adicionar as barras para vl_quantidade
fig.add_trace(go.Bar(
    x=df_v_f_anomes_agrupado['dt_fatura_data'],
    y=df_v_f_anomes_agrupado['vl_quantidade'],
    name='Quantidade Vendida',
    marker_color='blue'
))

# Adicionar a linha para vl_vendas_geral
fig.add_trace(go.Scatter(
    x=df_v_f_anomes_agrupado['dt_fatura_data'],
    y=df_v_f_anomes_agrupado['vl_vendas_geral'],
    name='Valor Total das Vendas',
    mode='lines',
    line=dict(color='red')
))

# Atualizar o layout do gráfico
fig.update_layout(
    title='Quantidade de Produtos Vendidos vs Valor Total das Vendas por Dia',
    xaxis_title='Dia',
    yaxis_title='Quantidade/Valor',
    barmode='group'
)

# Ajustar para que no gráfico tanto as colunas quanto as linhas apresentem o resultado sem casas decimais
fig.update_traces(
    hovertemplate='%{x}<br>Quantidade Vendida: %{y:.0f}<extra></extra>',
    selector=dict(type='bar')
)
fig.update_traces(
    hovertemplate='%{x}<br>Valor Total das Vendas: %{y:.0f}<extra></extra>',
    selector=dict(type='scatter')
)

# Exibir o gráfico
fig.show()

In [41]:
#Quantidade de Produtos Vendidos vs Valor Total das Vendas por semana.
# Agrupar os dados pela coluna dt_ano_mes e somar o valor das colunas vl_quantidade e vl_vendas_geral
df_v_f_anomes_agrupado = df_vendas_geral_agrupado.groupby('dt_dia_semana').agg({'vl_quantidade': 'sum', 'vl_vendas_geral': 'sum'}).reset_index()

# Converter 'dt_ano_mes' para string para melhor exibição
df_v_f_anomes_agrupado['dt_dia_semana'] = df_v_f_anomes_agrupado['dt_dia_semana'].astype(str)

# Criar o gráfico de barras e linhas com Plotly
fig = go.Figure()

# Adicionar as barras para vl_quantidade
fig.add_trace(go.Bar(
    x=df_v_f_anomes_agrupado['dt_dia_semana'],
    y=df_v_f_anomes_agrupado['vl_quantidade'],
    name='Quantidade Vendida',
    marker_color='blue'
))

# Adicionar a linha para vl_vendas_geral
fig.add_trace(go.Scatter(
    x=df_v_f_anomes_agrupado['dt_dia_semana'],
    y=df_v_f_anomes_agrupado['vl_vendas_geral'],
    name='Valor Total das Vendas',
    mode='lines',
    line=dict(color='red')
))

# Atualizar o layout do gráfico
fig.update_layout(
    title='Quantidade de Produtos Vendidos vs Valor Total das Vendas por Semana',
    xaxis_title='Semana',
    yaxis_title='Quantidade/Valor',
    barmode='group'
)

# Ajustar para que no gráfico tanto as colunas quanto as linhas apresentem o resultado sem casas decimais
fig.update_traces(
    hovertemplate='%{x}<br>Quantidade Vendida: %{y:.0f}<extra></extra>',
    selector=dict(type='bar')
)
fig.update_traces(
    hovertemplate='%{x}<br>Valor Total das Vendas: %{y:.0f}<extra></extra>',
    selector=dict(type='scatter')
)

# Exibir o gráfico
fig.show()

In [42]:
#Quantidade de Produtos Vendidos vs Valor Total das Vendas por hora
# Agrupar os dados pela coluna dt_ano_mes e somar o valor das colunas vl_quantidade e vl_vendas_geral
df_v_f_anomes_agrupado = df_vendas_geral_agrupado.groupby('dt_hora_dia').agg({'vl_quantidade': 'sum', 'vl_vendas_geral': 'sum'}).reset_index()

# Converter 'dt_ano_mes' para string para melhor exibição
df_v_f_anomes_agrupado['dt_hora_dia'] = df_v_f_anomes_agrupado['dt_hora_dia'].astype(str)

# Criar o gráfico de barras e linhas com Plotly
fig = go.Figure()

# Adicionar as barras para vl_quantidade
fig.add_trace(go.Bar(
    x=df_v_f_anomes_agrupado['dt_hora_dia'],
    y=df_v_f_anomes_agrupado['vl_quantidade'],
    name='Quantidade Vendida',
    marker_color='blue'
))

# Adicionar a linha para vl_vendas_geral
fig.add_trace(go.Scatter(
    x=df_v_f_anomes_agrupado['dt_hora_dia'],
    y=df_v_f_anomes_agrupado['vl_vendas_geral'],
    name='Valor Total das Vendas',
    mode='lines',
    line=dict(color='red')
))

# Atualizar o layout do gráfico
fig.update_layout(
    title='Quantidade de Produtos Vendidos vs Valor Total das Vendas por Hora',
    xaxis_title='Hora',
    yaxis_title='Quantidade/Valor',
    barmode='group'
)

# Ajustar para que no gráfico tanto as colunas quanto as linhas apresentem o resultado sem casas decimais
fig.update_traces(
    hovertemplate='%{x}<br>Quantidade Vendida: %{y:.0f}<extra></extra>',
    selector=dict(type='bar')
)
fig.update_traces(
    hovertemplate='%{x}<br>Valor Total das Vendas: %{y:.0f}<extra></extra>',
    selector=dict(type='scatter')
)

# Exibir o gráfico
fig.show()

In [43]:
#Construção do Gráfico de Dispersão. Objetivo é identificar Outliers na base que comprometa a sequência da análise.

# Agrupar pelo código 'cd_fatura' e somar apenas as colunas 'vl_quantidade' e 'vl_vendas_geral'
df_grouped = df_vendas_geral_agrupado.groupby('cd_fatura').agg({'vl_quantidade': 'sum', 'vl_vendas_geral': 'sum'}).reset_index()

# Criar o gráfico de dispersão com Plotly
fig = px.scatter(df_grouped, x='vl_quantidade', y='vl_vendas_geral',
                 title='Gráfico de Dispersão de Vendas')

# Mostrar o gráfico
fig.show()

In [ ]:
# Filtrar o df_vendas_geral_agrupado para vl_quantidade <= 5000
df_vendas_geral_agrupado = df_vendas_geral_agrupado[df_vendas_geral_agrupado['vl_quantidade'] <= 5000]


In [ ]:
#Criando um Histograma para identificar os grupos de quantidade de vendas de produto.
#Criando uma cópia do Dataset.
df_v_f_m = df_vendas_geral_agrupado.copy()

# Criar a coluna com a classificação dos valores de vl_vendas_geral
df_v_f_m['qtd_vendas_geral_grupo'] = (df_v_f_m['vl_quantidade'] // 100) * 100

# Criar o histograma com Plotly
fig = px.histogram(df_v_f_m, x='qtd_vendas_geral_grupo', nbins=100, title='Histograma | QTD de Produtos Vendidos por Compra')

# Atualizar o layout do gráfico
fig.update_layout(
    xaxis_title='Quantidade de Produtos Vendidos',
    yaxis_title='Frequência')

# Exibir o gráfico
fig.show()


In [ ]:
# Filtrar o df_vendas_geral_agrupado para vl_quantidade <= 5000
df_vendas_geral_agrupado = df_vendas_geral_agrupado[df_vendas_geral_agrupado['vl_quantidade'] <= 500]

In [ ]:
#Por cd_fatura quantos cd_produtos distintos são comprados?
df_v_p_d = df_vendas_geral.copy()

# Agrupar o df_v_p_d pelo cd_fatura e contar de forma distinta por cd_fatura o cd_produtos
df_v_p_d_agrupado = df_v_p_d.groupby(['dt_ano_mes', 'dt_fatura_data', 'dt_dia_semana', 'dt_fatura_hora', 'dt_hora_dia', 'cd_cliente', 'ds_pais', 'cd_fatura']).agg({'cd_produto': 'count', 'vl_vendas_geral': 'sum'}).reset_index()

# Agrupar o df_v_p_d_agrupado pelo cd_produto e contar por cd_fatura e valor médio de vendas.
df_v_p_d_agrupado_g_1 = df_v_p_d_agrupado.groupby('cd_produto').agg({'cd_fatura': 'count', 'vl_vendas_geral': 'mean'}).reset_index()

#Retornar quantidade de produtos até 10.
df_v_p_d_agrupado_g_1 = df_v_p_d_agrupado_g_1[df_v_p_d_agrupado_g_1['cd_produto'] <= 10]

# Visualizar o DataFrame agrupado
df_v_p_d_agrupado_g_1.head(10)

,cd_produto,cd_fatura,vl_vendas_geral
0,1,1214,420.236886
1,2,679,230.437909
2,3,594,310.761869
3,4,569,341.641564
4,5,602,369.394252
5,6,552,354.768967
6,7,544,389.876140
7,8,540,396.758278
8,9,538,410.087286
9,10,476,313.084223


----------------------------------------

Modelo - Preparação da Base.

----------------------------------------

In [ ]:
# Ajuste na base de dados para seguir a análise apenas com UK uma vez
# que ela representa 90,18% da base.
df_2 = df[df['ds_pais'] == 'United Kingdom']

# Filtro na base de dados para considerar a quantidade de produtos adquiridos
# igual ou inferior a 5 mil unidades.
df_modelo = df_2[df_2['vl_quantidade'] <= 5000]

#Agrupando df para cd_fatura.
df_modelo_agrupado = df_modelo.groupby(['dt_ano_mes', 'dt_fatura_data', 'dt_dia_semana', 'dt_fatura_hora', 'dt_hora_dia', 'cd_cliente', 'ds_pais', 'cd_fatura']).agg({'cd_produto': 'count', 'vl_quantidade':'sum', 'vl_vendas_geral': 'sum'}).reset_index()

#Visualizar o dataset.
df_modelo_agrupado.head()

,dt_ano_mes,dt_fatura_data,dt_dia_semana,dt_fatura_hora,dt_hora_dia,cd_cliente,ds_pais,cd_fatura,cd_produto,vl_quantidade,vl_vendas_geral
0,2010-12,2010-12-01,Quarta-feira,08:26:00,8,17850.0,United Kingdom,536365,7,40,139.12
1,2010-12,2010-12-01,Quarta-feira,08:28:00,8,17850.0,United Kingdom,536366,2,12,22.20
2,2010-12,2010-12-01,Quarta-feira,08:34:00,8,13047.0,United Kingdom,536367,12,83,278.73
3,2010-12,2010-12-01,Quarta-feira,08:34:00,8,13047.0,United Kingdom,536368,4,15,70.05
4,2010-12,2010-12-01,Quarta-feira,08:35:00,8,13047.0,United Kingdom,536369,1,3,17.85


In [ ]:
#Preparação para a construção do modelo.
df_modelo_1 = df_modelo_agrupado[['cd_fatura']]

#Criando a coluna flag modelo, com base em todos os filtros adotados anteriormente.
df_modelo_1['fl_modelo'] = 1

#Unindo a fl_modelo com o dataset original.
df_modelo_2 = pd.merge(df_modelo, df_modelo_1, on='cd_fatura')

#Filtrando o dataset final do modelo com base nos filtros adotados anteriormente. Como: UK, <= 500 compras por pedido.
df_modelo_3 = df_modelo_2[df_modelo_2['fl_modelo'] == 1]

#Visualizando o dataset.
df_modelo_3.head()

,dt_ano_mes,dt_fatura_data,dt_dia_semana,dt_fatura_hora,dt_hora_dia,cd_cliente,ds_pais,cd_fatura,cd_produto,ds_produto,vl_produto_unitario,vl_quantidade,vl_vendas_geral,fl_modelo
0,2010-12,2010-12-01,Quarta-feira,08:26:00,8,17850.0,United Kingdom,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,2.55,6,15.30,1
1,2010-12,2010-12-01,Quarta-feira,08:26:00,8,17850.0,United Kingdom,536365,71053,WHITE METAL LANTERN,3.39,6,20.34,1
2,2010-12,2010-12-01,Quarta-feira,08:26:00,8,17850.0,United Kingdom,536365,84406B,CREAM CUPID HEARTS COAT HANGER,2.75,8,22.00,1
3,2010-12,2010-12-01,Quarta-feira,08:26:00,8,17850.0,United Kingdom,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,3.39,6,20.34,1
4,2010-12,2010-12-01,Quarta-feira,08:26:00,8,17850.0,United Kingdom,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,3.39,6,20.34,1


----------------------------------------

Modelo

----------------------------------------

####Incluir aqui um exemplo do modelo Apriori.

In [ ]:
# Selecionar as colunas desejadas
df_3 = df_modelo_3[['cd_fatura', 'cd_produto']]
df_3.head()

,cd_fatura,cd_produto
0,536365,85123A
1,536365,71053
2,536365,84406B
3,536365,84029G
4,536365,84029E


In [ ]:
# Transformar os dados no formato apropriado para o modelo Apriori.

# Criação de matriz binaria para utilização posterior do algoritmo apriori.
basket = df_3.groupby(['cd_fatura', 'cd_produto']).size().unstack().reset_index().fillna(0).set_index('cd_fatura')
basket = basket.applymap(lambda x: 1 if x > 0 else 0)

#Visualizar o dataset.
basket.head()

cd_produto,10002,10080,10120,10125,10133,10135,11001,15030,15034,15036,...,90214O,90214P,90214R,90214S,90214T,90214U,90214V,90214W,90214Y,90214Z
cd_fatura,,,,,,,,,,,,,,,,,,,,,
536365,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
536366,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
536367,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
536368,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
536369,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
# Gerar conjuntos frequentes de itens usando o algoritmo Apriori
frequent_itemsets = apriori(basket, min_support=0.018, use_colnames=True)

# Gerar as regras de associação
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1)

#Visualizando o dataset.
rules.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,(20725),(20726),0.067563,0.040659,0.021958,0.325000,7.993361,0.019211,1.421246,0.938290
1,(20726),(20725),0.040659,0.067563,0.021958,0.540059,7.993361,0.019211,2.027297,0.911976
2,(20725),(20727),0.067563,0.060083,0.029197,0.432143,7.192402,0.025138,1.655199,0.923349
3,(20727),(20725),0.060083,0.067563,0.029197,0.485944,7.192402,0.025138,1.813880,0.916001
4,(20728),(20725),0.053206,0.067563,0.024914,0.468254,6.930577,0.021319,1.753537,0.903800


In [ ]:
#Arredondando o valor de efeito (Lift).
rules_3 = rules
rules_3['lift'] = rules_3['lift'].round(0)

#Visualizando o dataset.
rules_3.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,(20725),(20726),0.067563,0.040659,0.021958,0.325000,8.0,0.019211,1.421246,0.938290
1,(20726),(20725),0.040659,0.067563,0.021958,0.540059,8.0,0.019211,2.027297,0.911976
2,(20725),(20727),0.067563,0.060083,0.029197,0.432143,7.0,0.025138,1.655199,0.923349
3,(20727),(20725),0.060083,0.067563,0.029197,0.485944,7.0,0.025138,1.813880,0.916001
4,(20728),(20725),0.053206,0.067563,0.024914,0.468254,7.0,0.021319,1.753537,0.903800


In [ ]:
# Contar a frequência dos valores na coluna 'vl_efeito'
value_counts = rules_3['lift'].value_counts().reset_index()
value_counts.columns = ['lift', 'count']

# Ordenar o DataFrame pelos valores de 'vl_efeito'
value_counts = value_counts.sort_values(by='lift')

In [ ]:
# Criar o gráfico de barras com Plotly e incluir rótulos de dados
fig = px.bar(value_counts, x='lift', y='count',
             labels={'lift': 'lift', 'count': 'Contagem'},
             title='Distribuição do lift',
             text='count')

# Atualizar o layout do gráfico para mostrar os rótulos de dados
fig.update_traces(textposition='outside')

# Mostrar o gráfico
fig.show()

--------------------------------------------------------------

Análise dos Resultados

-------------------------------------------------------------

In [ ]:
rules_3.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,(20725),(20726),0.067563,0.040659,0.021958,0.325000,8.0,0.019211,1.421246,0.938290
1,(20726),(20725),0.040659,0.067563,0.021958,0.540059,8.0,0.019211,2.027297,0.911976
2,(20725),(20727),0.067563,0.060083,0.029197,0.432143,7.0,0.025138,1.655199,0.923349
3,(20727),(20725),0.060083,0.067563,0.029197,0.485944,7.0,0.025138,1.813880,0.916001
4,(20728),(20725),0.053206,0.067563,0.024914,0.468254,7.0,0.021319,1.753537,0.903800


In [ ]:
#Tratamento realizado para remover os () das colunas antecedents e consequents.
#Selecionando as colunas de produtos Antecedentes e Consequentes
df_recomendacao = rules_3[['antecedents', 'consequents']]

def remove_parentheses(value):
    # Se for um frozenset ou qualquer outro tipo de conjunto, converta para uma lista e remova os parênteses
    if isinstance(value, (frozenset, set, tuple)):
        return ','.join(map(str, value))  # converte em string e remove os parênteses
    return value

# Aplicando a função a cada célula do DataFrame
df_recomendacao['antecedents'] = df_recomendacao['antecedents'].apply(remove_parentheses)
df_recomendacao['consequents'] = df_recomendacao['consequents'].apply(remove_parentheses)

#Visualizando o dataset.
df_recomendacao.head()

,antecedents,consequents
0,20725,20726
1,20726,20725
2,20725,20727
3,20727,20725
4,20728,20725


--------------------------------------------------------------

Análise dos Resultados - Qual o incremento de Vendas?

-------------------------------------------------------------

In [ ]:
#Criando uma cópia do df para trabalhar.
df_estudo_final = df_modelo_3.copy()

#Visualizando o dataset.
df_estudo_final.head(1)

,dt_ano_mes,dt_fatura_data,dt_dia_semana,dt_fatura_hora,dt_hora_dia,cd_cliente,ds_pais,cd_fatura,cd_produto,ds_produto,vl_produto_unitario,vl_quantidade,vl_vendas_geral,fl_modelo
0,2010-12,2010-12-01,Quarta-feira,08:26:00,8,17850.0,United Kingdom,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,2.55,6,15.3,1


In [ ]:
# Agrupar por 'cd_produto' e calcular a média de 'vl_quantidade' e 'vl_produto_unitario'
df_produto_agrupado = df_estudo_final.groupby('cd_produto').agg(
    vl_quantidade=('vl_quantidade', 'mean'),
    vl_produto_unitario=('vl_produto_unitario', 'mean')
    ).reset_index()

# Ajustar 'vl_quantidade' para inteiro
df_produto_agrupado['vl_quantidade'] = df_produto_agrupado['vl_quantidade'].astype(int)

# Ajustar 'vl_produto_unitario' para duas casas decimais
#df_produto_agrupado['vl_produto_unitario'] = df_produto_agrupado['vl_produto_unitario'].round(2)

#Criando a coluna vl_compra_total
df_produto_agrupado['vl_compra_total'] = df_produto_agrupado['vl_quantidade'] * df_produto_agrupado['vl_produto_unitario']

df_produto_agrupado = df_produto_agrupado[['cd_produto', 'vl_compra_total']]

df_produto_agrupado = df_produto_agrupado.rename(columns={'cd_produto': 'cd_produto_rec', 'vl_compra_total': 'vl_compra_total_rec'})

# Exibir o resultado
df_produto_agrupado.head()

,cd_produto_rec,vl_compra_total_rec
0,10002,8.500000
1,10080,5.354762
2,10120,1.260000
3,10125,9.477600
4,10133,10.040261


In [ ]:
# Merge entre df_estudo_final e df_recomendacao
df_conso_1 = pd.merge(df_estudo_final, df_recomendacao, left_on='cd_produto', right_on='antecedents', how='left')

#Visualizando o dataset.
df_conso_1.head(3)

,dt_ano_mes,dt_fatura_data,dt_dia_semana,dt_fatura_hora,dt_hora_dia,cd_cliente,ds_pais,cd_fatura,cd_produto,ds_produto,vl_produto_unitario,vl_quantidade,vl_vendas_geral,fl_modelo,antecedents,consequents
0,2010-12,2010-12-01,Quarta-feira,08:26:00,8,17850.0,United Kingdom,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,2.55,6,15.30,1,85123A,21733
1,2010-12,2010-12-01,Quarta-feira,08:26:00,8,17850.0,United Kingdom,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,2.55,6,15.30,1,85123A,82482
2,2010-12,2010-12-01,Quarta-feira,08:26:00,8,17850.0,United Kingdom,536365,71053,WHITE METAL LANTERN,3.39,6,20.34,1,NaN,NaN


In [ ]:
#Transformando a coluna cd_produto_consequente(Consequents) no formato texto.
df_conso_1['consequents'] = df_conso_1['consequents'].astype(str)

#Transformando a coluna cd_produto_recorrente no formato texto.
df_produto_agrupado['cd_produto_rec'] = df_produto_agrupado['cd_produto_rec'].astype(str)


In [ ]:
#União entre as tabelas.
df_conso_2 = pd.merge(df_conso_1, df_produto_agrupado, left_on='consequents', right_on='cd_produto_rec', how='left')

#df_conso_2['vl_vendas_novo'] = df_conso_2['vl_vendas_geral'] + df_conso_2['vl_compra_total_rec']

#Visualizando o dataset.
df_conso_2.head()

,dt_ano_mes,dt_fatura_data,dt_dia_semana,dt_fatura_hora,dt_hora_dia,cd_cliente,ds_pais,cd_fatura,cd_produto,ds_produto,vl_produto_unitario,vl_quantidade,vl_vendas_geral,fl_modelo,antecedents,consequents,cd_produto_rec,vl_compra_total_rec
0,2010-12,2010-12-01,Quarta-feira,08:26:00,8,17850.0,United Kingdom,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,2.55,6,15.30,1,85123A,21733,21733,32.478214
1,2010-12,2010-12-01,Quarta-feira,08:26:00,8,17850.0,United Kingdom,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,2.55,6,15.30,1,85123A,82482,82482,23.898445
2,2010-12,2010-12-01,Quarta-feira,08:26:00,8,17850.0,United Kingdom,536365,71053,WHITE METAL LANTERN,3.39,6,20.34,1,NaN,nan,NaN,NaN
3,2010-12,2010-12-01,Quarta-feira,08:26:00,8,17850.0,United Kingdom,536365,84406B,CREAM CUPID HEARTS COAT HANGER,2.75,8,22.00,1,NaN,nan,NaN,NaN
4,2010-12,2010-12-01,Quarta-feira,08:26:00,8,17850.0,United Kingdom,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,3.39,6,20.34,1,NaN,nan,NaN,NaN


In [ ]:
# Função para substituir NaN por 0 em colunas específicas
def substituir_nan_por_zero_colunas(df, colunas):
    # Apenas substitui os NaN nas colunas especificadas
    df[colunas] = df[colunas].fillna(0)
    return df

#Relação de colunas para substituir o NaN por 0.
colunas_especificas = ['antecedents', 'consequents', 'cd_produto_rec', 'vl_compra_total_rec']

#Função.
df_novo = substituir_nan_por_zero_colunas(df_conso_2, colunas_especificas)

#Visualizando o dataset.
df_novo.head()

,dt_ano_mes,dt_fatura_data,dt_dia_semana,dt_fatura_hora,dt_hora_dia,cd_cliente,ds_pais,cd_fatura,cd_produto,ds_produto,vl_produto_unitario,vl_quantidade,vl_vendas_geral,fl_modelo,antecedents,consequents,cd_produto_rec,vl_compra_total_rec
0,2010-12,2010-12-01,Quarta-feira,08:26:00,8,17850.0,United Kingdom,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,2.55,6,15.30,1,85123A,21733,21733,32.478214
1,2010-12,2010-12-01,Quarta-feira,08:26:00,8,17850.0,United Kingdom,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,2.55,6,15.30,1,85123A,82482,82482,23.898445
2,2010-12,2010-12-01,Quarta-feira,08:26:00,8,17850.0,United Kingdom,536365,71053,WHITE METAL LANTERN,3.39,6,20.34,1,0,nan,0,0.000000
3,2010-12,2010-12-01,Quarta-feira,08:26:00,8,17850.0,United Kingdom,536365,84406B,CREAM CUPID HEARTS COAT HANGER,2.75,8,22.00,1,0,nan,0,0.000000
4,2010-12,2010-12-01,Quarta-feira,08:26:00,8,17850.0,United Kingdom,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,3.39,6,20.34,1,0,nan,0,0.000000


In [ ]:
# prompt: Preciso criar uma função para o df_novo onde caso a coluna consequents for maior que 0, marcar 1 na coluna fl_consequente. do contrario, 0.

def marcar_fl_consequente(df):
  """
  Cria uma nova coluna 'fl_consequente' no DataFrame, marcando 1 se 'consequents' for nan , caso contrário, 0.
  Args:
      df: O DataFrame a ser processado.
  Returns:
      O DataFrame com a nova coluna 'fl_consequente'.
  """
  df['fl_consequente'] = np.where(df['vl_compra_total_rec'] > 0, 1, 0)
  return df

# Aplicar a função ao DataFrame df_novo
df_novo = marcar_fl_consequente(df_novo)


In [ ]:
df_novo.head()

,dt_ano_mes,dt_fatura_data,dt_dia_semana,dt_fatura_hora,dt_hora_dia,cd_cliente,ds_pais,cd_fatura,cd_produto,ds_produto,vl_produto_unitario,vl_quantidade,vl_vendas_geral,fl_modelo,antecedents,consequents,cd_produto_rec,vl_compra_total_rec,fl_consequente
0,2010-12,2010-12-01,Quarta-feira,08:26:00,8,17850.0,United Kingdom,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,2.55,6,15.30,1,85123A,21733,21733,32.478214,1
1,2010-12,2010-12-01,Quarta-feira,08:26:00,8,17850.0,United Kingdom,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,2.55,6,15.30,1,85123A,82482,82482,23.898445,1
2,2010-12,2010-12-01,Quarta-feira,08:26:00,8,17850.0,United Kingdom,536365,71053,WHITE METAL LANTERN,3.39,6,20.34,1,0,nan,0,0.000000,0
3,2010-12,2010-12-01,Quarta-feira,08:26:00,8,17850.0,United Kingdom,536365,84406B,CREAM CUPID HEARTS COAT HANGER,2.75,8,22.00,1,0,nan,0,0.000000,0
4,2010-12,2010-12-01,Quarta-feira,08:26:00,8,17850.0,United Kingdom,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,3.39,6,20.34,1,0,nan,0,0.000000,0


In [ ]:
#Criando coluna para saber o valor de venda com a recomendação.
df_novo['vl_vendas_novo'] = df_novo['vl_vendas_geral'] + df_novo['vl_compra_total_rec']

In [ ]:
# Agrupar o df_v_p_d pelo cd_fatura e contar de forma distinta por cd_fatura o cd_produtos
df_conso_2_agrup = df_novo.groupby(['dt_ano_mes', 'dt_fatura_data', 'dt_dia_semana', 'dt_hora_dia', 'cd_cliente', 'ds_pais', 'cd_fatura']).agg({'cd_produto': 'count', 'fl_consequente': 'sum', 'vl_vendas_geral': 'sum', 'vl_compra_total_rec': 'sum', 'vl_vendas_novo': 'sum'}).reset_index()

# Visualizar o DataFrame agrupado
df_conso_2_agrup.head(20)

,dt_ano_mes,dt_fatura_data,dt_dia_semana,dt_hora_dia,cd_cliente,ds_pais,cd_fatura,cd_produto,fl_consequente,vl_vendas_geral,vl_compra_total_rec,vl_vendas_novo
0,2010-12,2010-12-01,Quarta-feira,8,13047.0,United Kingdom,536367,12,0,278.73,0.000000,278.730000
1,2010-12,2010-12-01,Quarta-feira,8,13047.0,United Kingdom,536368,4,0,70.05,0.000000,70.050000
2,2010-12,2010-12-01,Quarta-feira,8,13047.0,United Kingdom,536369,1,0,17.85,0.000000,17.850000
3,2010-12,2010-12-01,Quarta-feira,8,17850.0,United Kingdom,536365,8,2,154.42,56.376659,210.796659
4,2010-12,2010-12-01,Quarta-feira,8,17850.0,United Kingdom,536366,2,0,22.20,0.000000,22.200000
5,2010-12,2010-12-01,Quarta-feira,9,13748.0,United Kingdom,536371,1,0,204.00,0.000000,204.000000
6,2010-12,2010-12-01,Quarta-feira,9,14688.0,United Kingdom,536378,19,1,444.98,56.449373,501.429373
7,2010-12,2010-12-01,Quarta-feira,9,15100.0,United Kingdom,536374,1,0,350.40,0.000000,350.400000
8,2010-12,2010-12-01,Quarta-feira,9,15291.0,United Kingdom,536376,2,0,328.80,0.000000,328.800000
9,2010-12,2010-12-01,Quarta-feira,9,15311.0,United Kingdom,536381,35,0,449.98,0.000000,449.980000


--------------------------------------------------------------

Análise dos Resultados

-------------------------------------------------------------

In [ ]:
df_result_1 = df_conso_2_agrup.copy()

In [ ]:
df_result_2 = df_result_1[df_result_1['fl_consequente'] > 0]
df_result_2.head()

,dt_ano_mes,dt_fatura_data,dt_dia_semana,dt_hora_dia,cd_cliente,ds_pais,cd_fatura,cd_produto,fl_consequente,vl_vendas_geral,vl_compra_total_rec,vl_vendas_novo
3,2010-12,2010-12-01,Quarta-feira,8,17850.0,United Kingdom,536365,8,2,154.42,56.376659,210.796659
6,2010-12,2010-12-01,Quarta-feira,9,14688.0,United Kingdom,536378,19,1,444.98,56.449373,501.429373
10,2010-12,2010-12-01,Quarta-feira,9,16029.0,United Kingdom,536386,12,11,1993.20,386.180710,2379.380710
17,2010-12,2010-12-01,Quarta-feira,9,17850.0,United Kingdom,536373,17,3,275.16,80.275105,355.435105
18,2010-12,2010-12-01,Quarta-feira,9,17850.0,United Kingdom,536375,17,3,275.16,80.275105,355.435105


In [ ]:
# prompt: preciso de um group by no df_result_2 que retorne as colunas fl_consequente, fl_consequente	 contagem, vl_compra_total_rec média.

df_result_3 = df_result_2.groupby(['fl_consequente']) \
    .agg({'fl_consequente': 'count', 'vl_vendas_geral': 'mean', 'vl_compra_total_rec': 'mean'}) \
    .rename(columns={'fl_consequente': 'contagem'}) \
    .reset_index()

#Criando coluna para saber o valor de venda com a recomendação.
df_result_3['vl_vendas_geral_final'] = df_result_3['vl_vendas_geral'] + df_result_3['vl_compra_total_rec']

#Criando coluna para saber o valor de venda com a recomendação.
df_result_3['pc_incremento'] = (df_result_3['vl_compra_total_rec'] / df_result_3['vl_vendas_geral_final']) *100

df_result_3.head(50)


,fl_consequente,contagem,vl_vendas_geral,vl_compra_total_rec,vl_vendas_geral_final,pc_incremento
0,1,782,436.556240,37.676396,474.232636,7.944707
1,2,1389,567.868200,58.132432,626.000632,9.286322
2,3,261,531.658544,85.913432,617.571976,13.911485
3,4,53,531.028679,132.368424,663.397103,19.953121
4,5,6,225.048333,152.927228,377.975561,40.459554
5,6,3,806.883333,166.270055,973.153388,17.085699
6,7,1,295.990000,225.579351,521.569351,43.250116
7,8,1,1596.100000,338.550812,1934.650812,17.499324
8,10,671,1049.119091,329.731337,1378.850428,23.913496
9,11,379,1242.785567,383.947665,1626.733232,23.602374


In [ ]:
#Incremento na média por faixa de produto vendido a mais. (Ajustar os nomes das colunas)

df_result_4 = df_result_3.copy()

df_result_4 = df_result_4[df_result_4['contagem'] > 50]

df_result_4.head()

,fl_consequente,contagem,vl_vendas_geral,vl_compra_total_rec,vl_vendas_geral_final,pc_incremento
0,1,782,436.556240,37.676396,474.232636,7.944707
1,2,1389,567.868200,58.132432,626.000632,9.286322
2,3,261,531.658544,85.913432,617.571976,13.911485
3,4,53,531.028679,132.368424,663.397103,19.953121
8,10,671,1049.119091,329.731337,1378.850428,23.913496


In [ ]:
df_result_5 = df_result_4.copy()

df_result_5['vl_vendas_origin_total'] = df_result_5['contagem'] * df_result_5['vl_vendas_geral']
df_result_5['vl_vendas_incremento_total'] = df_result_5['contagem'] * df_result_5['vl_compra_total_rec']
df_result_5['vl_vendas_proposto_total'] = df_result_5['contagem'] * df_result_5['vl_vendas_geral_final']

df_result_5 = df_result_5[['fl_consequente', 'contagem', 'vl_vendas_origin_total', 'vl_vendas_incremento_total', 'vl_vendas_proposto_total']]

df_result_5['pc_incremento'] = (df_result_5['vl_vendas_incremento_total'] / df_result_5['vl_vendas_proposto_total']) *100

df_result_5['pc_incremento'] = df_result_5['pc_incremento'].round(2)
df_result_5['vl_vendas_origin_total'] = df_result_5['vl_vendas_origin_total'].round(2)
df_result_5['vl_vendas_incremento_total'] = df_result_5['vl_vendas_incremento_total'].round(2)
df_result_5['vl_vendas_proposto_total'] = df_result_5['vl_vendas_proposto_total'].round(2)

df_result_5.rename(columns={'fl_consequente': 'qtd_novos_prod_vendidos', 'contagem': 'qtd_clientes'}, inplace=True)

df_result_5.head(10)

,qtd_novos_prod_vendidos,qtd_clientes,vl_vendas_origin_total,vl_vendas_incremento_total,vl_vendas_proposto_total,pc_incremento
0,1,782,341386.98,29462.94,370849.92,7.94
1,2,1389,788768.93,80745.95,869514.88,9.29
2,3,261,138762.88,22423.41,161186.29,13.91
3,4,53,28144.52,7015.53,35160.05,19.95
8,10,671,703958.91,221249.73,925208.64,23.91
9,11,379,471015.73,145516.16,616531.89,23.60
10,12,272,361595.76,111911.77,473507.53,23.63
11,13,73,74421.47,32327.12,106748.59,30.28


--------------------------------------------------------------

Exportação de Dados

-------------------------------------------------------------

In [ ]:
# prompt: preciso substituir todos os . por , no df df_conso_2_agrup_novo

# Aplicar a função para substituir os pontos por vírgulas em todas as colunas do DataFrame
df_conso_2_agrup_novo = df_result_5.applymap(lambda x: str(x).replace('.', ','))


In [ ]:
#Agora, dada a regra de associação preciso medir, se eu recomendar e vender mais um produto, quanto eu ganho por esta venda?


# Exportar o DataFrame para um arquivo CSV
df_conso_2_agrup_novo.to_csv('df_fim_conso_1.8.csv', index=False, sep=';')

Tem alguma falha no modelo onde preciso revisitar a parte que traz o valor do produto recomendado. Está retornando apenas o valor unitário ou o valor médio dos produtos comprados?

4% de retorno do modelo é muito baixo.